In [1]:
require(pscl)
require(tibble)
library(tidyverse)

Loading required package: pscl

Classes and Methods for R originally developed in the
Political Science Computational Laboratory
Department of Political Science
Stanford University (2002-2015),
by and under the direction of Simon Jackman.
hurdle and zeroinfl functions by Achim Zeileis.

Loading required package: tibble

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ purrr     1.0.2
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [43]:
fake_cres<-read_csv("fake_cres.csv")

New names:
• `` -> `...1`


Rows: 14312 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): CRE, Cell_type
dbl (3): ...1, replicate_ID, umi_count

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [44]:
#definitely matters for this one
fake_cres$replicate_ID <- factor(fake_cres$replicate_ID)

#probably doesn't matter for these, but to be safe...
fake_cres$CRE <- factor(fake_cres$CRE)
fake_cres$Cell_type <- factor(fake_cres$Cell_type)

In [45]:
head(fake_cres)

...1,CRE,Cell_type,replicate_ID,umi_count
<dbl>,<fct>,<fct>,<fct>,<dbl>
0,nobody,brain,1,0
1,nobody,brain,1,0
2,nobody,brain,1,0
3,nobody,brain,1,0
4,nobody,brain,1,0
5,nobody,brain,1,0


```
zinb_logit <- zeroinfl(UMIs_mBC ~ rep_id + CRE_id | rep_id,
               dist = 'negbin',
               data = data_cardio, link='logit')
```

In [47]:
zinb <- zeroinfl(umi_count ~ CRE*Cell_type +0|replicate_ID,
    dist='negbin',
    data=fake_cres)

# in NB estimation no intercept makes the coef. more interpretable
# It also makes logical sense : No CREs means no expression !
# CRE alone : inherent CRE strength (like TFs which are in ALL cell types). 
# Cell-type alone : probably how much RNA is in that cell-type
# CRE x cell-type: how cell-type modifies CRE activity (what TFs are ONLY in that cell type)
# note that umi_count ~ CRE:`Cell-type` alone fails to converge

In [48]:
zinb


Call:
zeroinfl(formula = umi_count ~ CRE * Cell_type + 0 | replicate_ID, data = fake_cres, 
    dist = "negbin")

Count model coefficients (negbin with log link):
               CREeverybody                 CREneurogene  
                    4.64219                      2.80138  
                  CREnobody                   CREredgene  
                    0.62984                      4.63772  
                CREsomebody               Cell_typebrain  
                    2.56032                      0.03802  
CREneurogene:Cell_typebrain     CREnobody:Cell_typebrain  
                    1.71354                     -0.64983  
  CREredgene:Cell_typebrain   CREsomebody:Cell_typebrain  
                   -1.22715                     -0.28217  
Theta = 3.0508 

Zero-inflation model coefficients (binomial with logit link):
  (Intercept)  replicate_ID2  replicate_ID3  
       1.3499        -1.3278         0.8831  


Ok, let's start with the zero-inflation parameter, since that's simpler. 

logit(binomial parameter)=intercept + bool_id2 * (-1.3278) + bool_id3 * (0.8831)

Parameter is parameter of binomial distribution. Assuming "success" means "not a zero", then zero inflation as picked in data-creation will be 1-binomial.

(If "success" is "is a zero" then we can skip the 1- part. )

In [49]:
typeof(zinb)

[1] "list"

In [50]:
ls(zinb)

[1] "call"          "coefficients"  "contrasts"     "control"      
 [5] "converged"     "df.null"       "df.residual"   "dist"         
 [9] "fitted.values" "formula"       "levels"        "link"         
[13] "linkinv"       "loglik"        "method"        "model"        
[17] "n"             "offset"        "optim"         "residuals"    
[21] "SE.logtheta"   "start"         "terms"         "theta"        
[25] "vcov"          "weights"       "y"

In [51]:
zinb[["coefficients"]]

$count
               CREeverybody                CREneurogene 
                 4.64218549                  2.80137858 
                  CREnobody                  CREredgene 
                 0.62984483                  4.63771831 
                CREsomebody              Cell_typebrain 
                 2.56031912                  0.03801549 
CREneurogene:Cell_typebrain    CREnobody:Cell_typebrain 
                 1.71354030                 -0.64982606 
  CREredgene:Cell_typebrain  CREsomebody:Cell_typebrain 
                -1.22715075                 -0.28217018 

$zero
  (Intercept) replicate_ID2 replicate_ID3 
    1.3498616    -1.3277722     0.8831298

In [52]:
zinb[["coefficients"]][["zero"]]

(Intercept) replicate_ID2 replicate_ID3 
    1.3498616    -1.3277722     0.8831298

In [75]:
#OK, let's make a little matrix of all possibilities...
zeroinfl<-matrix(c(1,1,1,0,1,0,0,0,1),nrow=3,ncol=3)

rownames(zeroinfl)=levels(fake_cres$replicate_ID)
colnames(zeroinfl)<-names(zinb[["coefficients"]][["zero"]])

t(zeroinfl)

,1,2,3
(Intercept),1,1,1
replicate_ID2,0,1,0
replicate_ID3,0,0,1


In [79]:
weighted=zinb[["coefficients"]][["zero"]]*t(zeroinfl)#*matrix(zeroinfl,nrow=3,ncol=1)
weighted

,1,2,3
(Intercept),1.349862,1.349862,1.3498616
replicate_ID2,0.000000,-1.327772,0.0000000
replicate_ID3,0.000000,0.000000,0.8831298


In [83]:
logited_binom=colSums(weighted)
logited_binom

1          2          3 
1.34986164 0.02208947 2.23299139

In [85]:
invlogit <- function(x) {
  return(1 / (1 + exp(-x)))
}
invlogit(logited_binom)

1         2         3 
0.7941070 0.5055221 0.9031733

Comparing to , 